## Model Start & Stop Function

In [26]:
import boto3

def start_model(project_arn, model_arn, version_name, min_inference_units):

    client=boto3.client('rekognition')

    try:
        # Start the model
        print('Starting model: ' + model_arn)
        response=client.start_project_version(ProjectVersionArn=model_arn, MinInferenceUnits=min_inference_units)
        # Wait for the model to be in the running state
        project_version_running_waiter = client.get_waiter('project_version_running')
        project_version_running_waiter.wait(ProjectArn=project_arn, VersionNames=[version_name])

        #Get the running status
        describe_response=client.describe_project_versions(ProjectArn=project_arn,
            VersionNames=[version_name])
        for model in describe_response['ProjectVersionDescriptions']:
            print("Status: " + model['Status'])
            print("Message: " + model['StatusMessage']) 
    except Exception as e:
        print(e)
        
    print('Done...')

def stop_model(model_arn):

    client=boto3.client('rekognition')

    print('Stopping model:' + model_arn)

    #Stop the model
    try:
        response=client.stop_project_version(ProjectVersionArn=model_arn)
        status=response['Status']
        print ('Status: ' + status)
    except Exception as e:  
        print(e)  

    print('Done...')

### Model Start & Stop

In [29]:
import boto3

project_arn='arn:aws:rekognition:us-east-1:216064490951:project/kumho-logo/1630850112761'
model_arn='arn:aws:rekognition:us-east-1:216064490951:project/kumho-logo/version/kumho-logo.2021-09-05T22.55.46/1630850147101'
min_inference_units=2 
version_name='kumho-logo.2021-09-05T22.55.46'

start_model(project_arn, model_arn, version_name, min_inference_units)

#stop_model(model_arn)


Starting model: arn:aws:rekognition:us-east-1:216064490951:project/kumho-logo/version/kumho-logo.2021-09-05T22.55.46/1630850147101
Status: RUNNING
Message: The model is running.
Done...


### Model Run

In [32]:
import boto3 
import time 

client = boto3.client('rekognition')

bucket='test-rekog-unit'
photo='kumho-img.jpg'
model='arn:aws:rekognition:us-east-1:216064490951:project/kumho-logo/version/kumho-logo.2021-09-05T22.55.46/1630850147101'
min_confidence=95

start_time = time.time()

response = client.detect_custom_labels(Image={'S3Object': {'Bucket': bucket, 'Name': photo}},
    MinConfidence=min_confidence,
    ProjectVersionArn=model)

print("-- %s seconds ---"%(time.time() - start_time))

print(response)

-- 4.043249130249023 seconds ---
{'CustomLabels': [{'Name': 'kumho', 'Confidence': 98.09700012207031, 'Geometry': {'BoundingBox': {'Width': 0.43672001361846924, 'Height': 0.19670000672340393, 'Left': 0.326200008392334, 'Top': 0.16797000169754028}}}], 'ResponseMetadata': {'RequestId': 'dbeef593-0fee-4be1-863c-9c3f94884b0b', 'HTTPStatusCode': 200, 'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1', 'date': 'Sun, 03 Oct 2021 06:06:32 GMT', 'x-amzn-requestid': 'dbeef593-0fee-4be1-863c-9c3f94884b0b', 'content-length': '203', 'connection': 'keep-alive'}, 'RetryAttempts': 0}}


In [50]:
## 파일을 S3 로 업로드 이후에 모델을 Running
import boto3 
import time 

client = boto3.client('rekognition')
s3client = boto3.client('s3')
bucket='test-rekog-unit'
krbucket='kr-bucket'
photo='kumho-img-resize.jpg'

start_time = time.time()
s3client.upload_file('/Users/koonyoun/Documents/Dev/aws/kumho-img-resize.jpg', bucket, photo)
print("-- Upload file : %s seconds ---"%(time.time() - start_time))

start_time = time.time()
response = client.detect_custom_labels(Image={'S3Object': {'Bucket': bucket, 'Name': photo}},
    MinConfidence=min_confidence,
    ProjectVersionArn=model)


print("-- Inference : %s seconds ---"%(time.time() - start_time))

print(response)

-- Upload file : 4.6467859745025635 seconds ---
-- Inference : 3.5318832397460938 seconds ---
{'CustomLabels': [{'Name': 'kumho', 'Confidence': 98.00800323486328, 'Geometry': {'BoundingBox': {'Width': 0.42236000299453735, 'Height': 0.20058000087738037, 'Left': 0.3391599953174591, 'Top': 0.16707000136375427}}}], 'ResponseMetadata': {'RequestId': '6f02a5a2-c377-42de-8148-a81be152778f', 'HTTPStatusCode': 200, 'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1', 'date': 'Sun, 03 Oct 2021 07:52:28 GMT', 'x-amzn-requestid': '6f02a5a2-c377-42de-8148-a81be152778f', 'content-length': '204', 'connection': 'keep-alive'}, 'RetryAttempts': 0}}


In [44]:
## 파일을 Byte-stream 으로 전달
import boto3
import time 

client = boto3.client('rekognition')
start_time = time.time()

image = open('/Users/koonyoun/Documents/Dev/aws/kumho-img.jpg', 'rb')

response = client.detect_custom_labels(Image={'Bytes': image.read()},
    MinConfidence=min_confidence,
    ProjectVersionArn=model)


print("-- %s seconds ---"%(time.time() - start_time))

print(response)

-- 7.215771913528442 seconds ---
{'CustomLabels': [{'Name': 'kumho', 'Confidence': 98.09700012207031, 'Geometry': {'BoundingBox': {'Width': 0.43672001361846924, 'Height': 0.19670000672340393, 'Left': 0.326200008392334, 'Top': 0.16797000169754028}}}], 'ResponseMetadata': {'RequestId': '09d1df91-f89e-47ed-84ef-8f5f9a116681', 'HTTPStatusCode': 200, 'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1', 'date': 'Sun, 03 Oct 2021 06:11:11 GMT', 'x-amzn-requestid': '09d1df91-f89e-47ed-84ef-8f5f9a116681', 'content-length': '203', 'connection': 'keep-alive'}, 'RetryAttempts': 0}}


In [54]:
## 파일을 일반 상태로 Upload
import boto3
import time 

client = boto3.client('rekognition')
s3client = boto3.client('s3')
bucket='test-rekog-unit'
krbucket='kr-bucket'
photo='kumho-img-resize.jpg'

client = boto3.client('rekognition')
start_time = time.time()

response = client.detect_custom_labels(Image={'S3Object': {'Bucket': bucket, 'Name': photo}},
    MinConfidence=min_confidence,
    ProjectVersionArn=model)


print("-- %s seconds ---"%(time.time() - start_time))

print(response)


-- 3.4851441383361816 seconds ---
{'CustomLabels': [{'Name': 'kumho', 'Confidence': 98.00800323486328, 'Geometry': {'BoundingBox': {'Width': 0.42236000299453735, 'Height': 0.20058000087738037, 'Left': 0.3391599953174591, 'Top': 0.16707000136375427}}}], 'ResponseMetadata': {'RequestId': '70a039ea-0ef8-46f5-9382-88048cab0764', 'HTTPStatusCode': 200, 'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1', 'date': 'Sun, 03 Oct 2021 07:53:57 GMT', 'x-amzn-requestid': '70a039ea-0ef8-46f5-9382-88048cab0764', 'content-length': '204', 'connection': 'keep-alive'}, 'RetryAttempts': 0}}


In [40]:
!aws rekognition detect-custom-labels --project-version-arn "arn:aws:rekognition:us-east-1:216064490951:project/kumho-logo/version/kumho-logo.2021-09-05T22.55.46/1630850147101" --image '{"S3Object": {"Bucket": "test-rekog-unit","Name": "kumho-img.jpg"}}' 

{
    "CustomLabels": [
        {
            "Name": "kumho",
            "Confidence": 98.09700012207031,
            "Geometry": {
                "BoundingBox": {
                    "Width": 0.43672001361846924,
                    "Height": 0.19670000672340393,
                    "Left": 0.326200008392334,
                    "Top": 0.16797000169754028
                }
            }
        }
    ]
}


In [42]:
!aws rekognition detect-custom-labels --project-version-arn "arn:aws:rekognition:us-east-1:216064490951:project/kumho-logo/version/kumho-logo.2021-09-05T22.55.46/1630850147101" --image-bytes fileb://kumho-img.jpg

{
    "CustomLabels": [
        {
            "Name": "kumho",
            "Confidence": 98.09700012207031,
            "Geometry": {
                "BoundingBox": {
                    "Width": 0.43672001361846924,
                    "Height": 0.19670000672340393,
                    "Left": 0.326200008392334,
                    "Top": 0.16797000169754028
                }
            }
        }
    ]
}


In [21]:
print(boto3.__version__)

1.17.53


In [25]:
import boto3 
import time 

client = boto3.client('rekognition')
s3client = boto3.client('s3')
bucket='test-rekog-unit'
krbucket='kr-bucket'
photo='kumho-img.jpg'

start_time = time.time()
s3client.upload_file('/Users/koonyoun/Documents/Dev/aws/kumho-img.jpg', bucket, 'kumho-img.jpg')
print("-- Upload file : %s seconds ---"%(time.time() - start_time))

-- Upload file : 11.020255088806152 seconds ---
